In [3]:
#Nome: João Vitor Fidelis Cardozo, RA: 769719

In [9]:
import numpy as np
import random
import operator

Um elemento de uma matriz é definido por Aij e, portanto, define-se i origem e j destino e, sendo assim, Aij, expressa o custo (distância) de ir da cidade i para a cidade j. Como restrição temos que 0<=i<=5 e 0<=j<=5.

In [10]:
#Gera distâncias aleatórias
def gera_distancias_aleatorias():
    distancias = [[0 for coluna in range(6)] for linha in range(6)] 
    for linha in range(len(distancias)):
        for coluna in range(len(distancias)):
            if (linha == coluna):
                distancias[linha][coluna] = 0
            elif (coluna < linha):
                distancias[linha][coluna] = random.randint(0, 1000)
    return distancias

#A função abaixo é responsável por preencher a parte superior da matriz
def atualiza_matriz_distancias(distancias):
    for linha in range(len(distancias)):
        for coluna in range (len(distancias)):
            distancias[linha][coluna] = distancias[coluna][linha]
    return distancias

def exibe_matriz_distancias(distancias):
    for i in range(len(distancias)):
        for j in range(len(distancias)):
            print(f'{distancias[i][j]} ', end='')
        print("\n")

In [183]:
#Garante que todos os caminhos comecem na cidade A e terminem na cidade F
def geraCaminhoAleatorio(cidades):
    caminho = random.sample(cidades[1:5], len(cidades)-2)
    caminho.append("F")
    caminho.insert(0, "A")
    return caminho

#Gera a populacao inicial (matriz) com base na populacao acima
def populacaoInicial(tamanho, cidades):
    populacao = []
    
    for i in range(tamanho):
        populacao.append(geraCaminhoAleatorio(cidades))
    return populacao

#Retorna a distancia entre duas cidades consecutivas no vetor
def calculaDistancia(origem, dest, dict_cidades, distancias):
    distancia = distancias[dict_cidades[origem]][dict_cidades[dest]]
    return distancia

#Retorna a distancia total em um individuo
def fitness(dict_cidades, distancias, caminho):
    distancia_total = 0
    for cidade in range(len(caminho)):
        if cidade+1 < len(caminho):
            origem = caminho[cidade]
            destino = caminho[cidade+1]
            distancia_total += calculaDistancia(origem, destino, dict_cidades, distancias)
    distancia_total = distancia_total + calculaDistancia(caminho[len(caminho)-1], caminho[0], dict_cidades, distancias)
    return distancia_total


#Ordenar pelos individuos mais aptos (menor distancia)
def elitismo(populacao, dict_cidades, distancias):
    mais_aptos = {}
    for individuo in range(len(populacao)):
        mais_aptos[individuo] = fitness(dict_cidades, distancias, populacao[individuo]) 
    return sorted(mais_aptos.items(), key=operator.itemgetter(1), reverse=False)
    
#Seleciona os dois melhores individuos com base no valor de seus fitness
#Garante a seleção de individuos não repetidos
def selecao_por_roleta(dict_melhores, populacao):
    parent1 = populacao[dict_melhores[0][0]]
    parent2 = populacao[dict_melhores[1][0]]
    while (parent1 == parent2):
        for i in range(len(dict_melhores)):
            parent2 = populacao[dict_melhores[i][0]]
    return parent1, parent2

#Cruza os dois melhores individuos
def crossover(pai1, pai2):
    parent1 = pai1.copy()[1:5]
    parent2 = pai2.copy()[1:5]

    filho1 = []
    filho2 = []
    
    filho1P1 = []
    filho1P2 = []
    
    filho2P1 = []
    filho2P2 = []

    inicioGene = np.random.randint(1, len(parent1))
    fimGene = np.random.randint(1, len(parent1))
    while inicioGene == fimGene:
        fimGene = np.random.randint(1, len(parent1))

    for i in range(min(inicioGene, fimGene), max(inicioGene, fimGene)):
        filho1P1.append(parent1[i])
        filho2P1.append(parent2[i])
    filho1P2 = [item for item in parent2 if item not in filho1P1]
    filho2P2 = [item for item in parent1 if item not in filho2P1]
    filho1 = filho1P1 + filho1P2
    filho2 = filho2P1 + filho2P2
    filho1.append("F")
    filho1.insert(0, "A")
    filho2.append("F")
    filho2.insert(0, "A")
    return filho1, filho2

#Mutação de um individuo com base em uma taxa
def mutacao(filhos, taxa):
    for i in range(len(filhos)):
        if random.random() < taxa:
            pos_inicio = random.randint(1,4)
            pos_fim = random.randint(1,4)
            while (pos_inicio == pos_fim):
                pos_fim = random.randint(1,4)
                aux_troca = filhos[i][pos_inicio]
                filhos[i][pos_inicio] = filhos[i][pos_fim]
                filhos[i][pos_fim] = aux_troca
    return filhos

def exibeDados(populacao, num_individuos, dict_cidades, distancias):
    melhor_dict = elitismo(populacao, dict_cidades, distancias)
    melhor = populacao[melhor_dict[0][0]]
    for i in range(num_individuos):
        print(f'Caminho:{populacao[i]}+"A", distancia:{fitness(dict_cidades, distancias, populacao[i])}')
    print(f'Melhor caminho:{melhor}+"A", melhor distancia:{fitness(dict_cidades, distancias, melhor)}')

In [182]:
#Definição da matriz de distâncias
distancias = gera_distancias_aleatorias()
distancias = atualiza_matriz_distancias(distancias)
#exibe_matriz_distancias(distancias)

#Definição das cidades
cidades = ["A", "B", "C", "D", "E", "F"]
dict_cidades = {"A":0, "B":1, "C":2, "D":3, "E":4, "F":5}

#Definição do número de gerações
num_geracoes = 20

#Número de individuos
num_individuos = 10

#Populacao inicial
populacao_inicial = populacaoInicial(num_individuos, cidades)

for iteracao in range(num_geracoes):
    #Gera nova populacao
    nova_populacao = [["N" for coluna in range(6)] for linha in range(num_individuos)]
    #Exibe caminhos e distancias
    exibeDados(populacao_inicial, num_individuos, dict_cidades, distancias)
    print("\n")
    
    #Elitismo
    dict_melhores = elitismo(populacao_inicial, dict_cidades, distancias)
    
    elite = selecao_por_roleta(dict_melhores, populacao_inicial)
    nova_populacao[0] = elite[0]
    nova_populacao[1] = elite[1]
    
    num_filhos = 2
    while (num_filhos < num_individuos):
        #Cruzamento
        filhos = crossover(elite[0], elite[1])
        
        #Mutação (Arrumar)
        filhos = mutacao(filhos, 0.7)
        
        #Colocar os filhos na nova população
        nova_populacao[num_filhos] = filhos[0]
        nova_populacao[num_filhos+1] = filhos[1]
        
        #Aumenta o número de filhos
        num_filhos = num_filhos + 2
        #print(nova_populacao)
    populacao_inicial = nova_populacao.copy()

Caminho:['A', 'C', 'E', 'B', 'D', 'F']+"A", distancia:3238
Caminho:['A', 'E', 'C', 'D', 'B', 'F']+"A", distancia:4062
Caminho:['A', 'D', 'C', 'E', 'B', 'F']+"A", distancia:3252
Caminho:['A', 'E', 'D', 'B', 'C', 'F']+"A", distancia:3320
Caminho:['A', 'B', 'C', 'E', 'D', 'F']+"A", distancia:3414
Caminho:['A', 'E', 'D', 'C', 'B', 'F']+"A", distancia:3216
Caminho:['A', 'B', 'C', 'E', 'D', 'F']+"A", distancia:3414
Caminho:['A', 'C', 'B', 'D', 'E', 'F']+"A", distancia:2602
Caminho:['A', 'D', 'C', 'B', 'E', 'F']+"A", distancia:2300
Caminho:['A', 'B', 'E', 'C', 'D', 'F']+"A", distancia:2960
Melhor caminho:['A', 'D', 'C', 'B', 'E', 'F']+"A", melhor distancia:2300


Caminho:['A', 'D', 'C', 'B', 'E', 'F']+"A", distancia:2300
Caminho:['A', 'C', 'B', 'D', 'E', 'F']+"A", distancia:2602
Caminho:['A', 'B', 'C', 'D', 'E', 'F']+"A", distancia:2324
Caminho:['A', 'D', 'C', 'B', 'E', 'F']+"A", distancia:2300
Caminho:['A', 'B', 'C', 'D', 'E', 'F']+"A", distancia:2324
Caminho:['A', 'D', 'C', 'E', 'B', 'F']+"